<a href="https://colab.research.google.com/github/dataplusplus-ai/Computer_Vision_Deep_Learning/blob/main/zindi_transferlearning_ResNet101v2_imagenet_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#resnet with transfer learning

In [ ]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/zindi_train.zip"
!unzip -q "/content/gdrive/My Drive/zindi_test.zip"
!unzip -q "/content/gdrive/My Drive/zindi_validation.zip"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
%tensorflow_version 1.x

import numpy as np
import pandas as pd
import cv2
import json

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from __future__ import print_function
import keras
from keras import models
from keras import layers
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPool2D , Flatten
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

import os
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import plot_model

Using TensorFlow backend.


In [ ]:
from PIL import Image
import os

In [ ]:
def convert_jpg(old_dir,new_dir):
  for filename in os.listdir(old_dir):
    if filename.lower().endswith(".jfif") or filename.lower().endswith(".jpg") or filename.lower().endswith(".png") or filename.lower().endswith(".jpeg"):
        im=Image.open(old_dir+filename)
        name=filename.split('.')[0]+'.jpg'
        im.save(new_dir+name)

In [ ]:
dir1='/content/train/healthy_wheat/'
dir2='/content/trainjpg/healthy_wheat/'
convert_jpg(dir1,dir2)

In [ ]:
dir1='/content/train/leaf_rust/'
dir2='/content/trainjpg/leaf_rust/'
convert_jpg(dir1,dir2)

In [ ]:
dir1='/content/train/stem_rust/'
dir2='/content/trainjpg/stem_rust/'
convert_jpg(dir1,dir2)

In [ ]:

#test images are converted to jpgs
dir1='/content/test/'
dir2='/content/testjpg/'
convert_jpg(dir1,dir2)

In [ ]:
#convert images in validation set to jpegs
dir1='/content/validation/healthy_wheat_val/'
dir2='/content/validationjpg/healthy_wheat/'
convert_jpg(dir1,dir2)

In [ ]:
dir1='/content/validation/leaf_rust_val/'
dir2='/content/validationjpg/leaf_rust/'
convert_jpg(dir1,dir2)

In [ ]:
dir1='/content/validation/stem_rust_val/'
dir2='/content/validationjpg/stem_rust/'
convert_jpg(dir1,dir2)

In [ ]:
print('no of test images is {}'.format(len(os.listdir('/content/testjpg'))))
print('no of train images is {}+{}+{}'.format(len(os.listdir('/content/trainjpg/healthy_wheat')),
                                               len(os.listdir('/content/trainjpg/stem_rust')),
                                              len(os.listdir('/content/trainjpg/leaf_rust'))))
print('no of validation images is {}+{}+{}'.format(len(os.listdir('/content/validationjpg/healthy_wheat')),
                                               len(os.listdir('/content/validationjpg/stem_rust')),
                                              len(os.listdir('/content/validationjpg/leaf_rust'))))

no of test images is 610
no of train images is 130+364+345
no of validation images is 12+12+12


In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):

  """
  p : the probability that random erasing is performed
  s_l, s_h : minimum / maximum proportion of erased area against input image
  r_1, r_2 : minimum / maximum aspect ratio of erased area
  v_l, v_h : minimum / maximum value for erased area
  pixel_level : pixel-level randomization for erased area
  """

  def eraser(input_img):
    img_h, img_w, img_c = input_img.shape
    p_1 = np.random.rand()

    if p_1 > p:
        return input_img

    while True:
        s = np.random.uniform(s_l, s_h) * img_h * img_w
        r = np.random.uniform(r_1, r_2)
        w = int(np.sqrt(s / r))
        h = int(np.sqrt(s * r))
        left = np.random.randint(0, img_w)
        top = np.random.randint(0, img_h)

        if left + w <= img_w and top + h <= img_h:
            break

    if pixel_level:
        c = np.random.uniform(v_l, v_h, (h, w, img_c))
    else:
        c = np.random.uniform(v_l, v_h)

    input_img[top:top + h, left:left + w, :] = c

    return input_img

  return eraser

In [ ]:
from imgaug import augmenters as iaa

aug1 = iaa.CoarseDropout(p=0.10, size_percent=0.05)

def additional_augmenation(image):
    image = aug1.augment_image(image)

    return image

In [ ]:
img_width, img_height = 224,224
batch_size=32
train_datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=2,
        # randomly shift images horizontally
        width_shift_range=0.08,
        # randomly shift images vertically
        height_shift_range=0.05,
        # set range for random shear
        shear_range=0.07,
        # set range for random zoom
        zoom_range=0.05,
        # set range for random channel shifts
        channel_shift_range=0.07,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        rescale=1. / 255,
        # set rescaling factor (applied before any other transformation)
        # set function that will be applied on each input
        #Cutout Implementation
        preprocessing_function=get_random_eraser(p=0.08,v_l=0, v_h=0.3, s_l=0.008, s_h=0.05, pixel_level=False),
        #preprocessing_function=additional_augmenation, 
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

In [ ]:
#behaves weird, create a folder in content, move trainjpg there. Then create trainjpg here in content adn move from there
train_generator = train_datagen.flow_from_directory(
    directory=r"/content/trainjpg/tr/",
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 839 images belonging to 3 classes.


In [ ]:
#finall y working
validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_directory(
    directory=r"/content/val/validationjpg/",
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 36 images belonging to 3 classes.


In [ ]:
in_shape=train_generator.__getitem__(16)[0][1].shape
in_shape

(224, 224, 3)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/pred/",
    target_size=(224,224),
    batch_size=1,
    class_mode=None,
    shuffle=False)

Found 610 images belonging to 1 classes.


In [ ]:
def lr_schedule(epoch):

    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1

    print('Learning Rate: ', lr)
    return lr

In [ ]:

from keras.callbacks import Callback
from keras import backend as K
import numpy as np

# Keras-Contib Implementation
class CyclicLR(Callback):

    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.004,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2.**(x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        print("Learning Rate: ", float(K.get_value(self.model.optimizer.lr)))
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
# Prepare callbacks for model saving and for learning rate adjustment.

#filepath='/content/gdrive/My Drive/resnet.h5'

#checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc', verbose=1, save_best_only=True)
checkpoint = ModelCheckpoint("densenet.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
clr = CyclicLR(base_lr=0.001, 
               max_lr=0.004, # chnged to .004 from .005
               step_size=2000., 
               mode='triangular')

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.4e-6)  #chnged to .4 from .5

#callbacks = [checkpoint, lr_reducer, clr, plot, save_weight]
callbacks = [lr_reducer,clr]


NameError: ignored

In [ ]:
from keras import models
from keras import layers

In [ ]:
from keras.applications import ResNet101V2
conv_base=ResNet101V2(weights='imagenet',include_top=False,input_shape=(224,224,3))













171319296/171317808 [==============================] - 14s 0us/step


In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
model=conv_base.output
model=GlobalAveragePooling2D()(model)

predictions = Dense(3, activation= 'softmax')(model)
model = Model(inputs = conv_base.input, outputs = predictions)

In [ ]:
conv_base.trainable=False

In [ ]:
# compile set call backs
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [ ]:
nb_epoch=200
last_executed_epoch=0

print('Training for Epochs : ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " -------------------------------")

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(generator=train_generator, validation_data=validation_generator, use_multiprocessing=True, epochs=nb_epoch, initial_epoch=last_executed_epoch, verbose=1, workers=6, callbacks=callbacks)

Training for Epochs : 1 - 200 -------------------------------
Epoch 1/200
27/27 [==============================] - 67s 2s/step - loss: 0.0533 - acc: 0.9826 - val_loss: 0.5315 - val_acc: 0.8889
Learning Rate:  0.003700000001117587
Epoch 2/200
27/27 [==============================] - 65s 2s/step - loss: 0.0665 - acc: 0.9763 - val_loss: 1.1263 - val_acc: 0.8333
Learning Rate:  0.0037674999330192804
Epoch 3/200
27/27 [==============================] - 64s 2s/step - loss: 0.0473 - acc: 0.9803 - val_loss: 0.5852 - val_acc: 0.8611
Learning Rate:  0.0038350000977516174
Epoch 4/200
27/27 [==============================] - 63s 2s/step - loss: 0.0541 - acc: 0.9780 - val_loss: 0.6099 - val_acc: 0.8056
Learning Rate:  0.0039025000296533108
Epoch 5/200
27/27 [==============================] - 64s 2s/step - loss: 0.0854 - acc: 0.9710 - val_loss: 1.1181 - val_acc: 0.7778
Learning Rate:  0.003969999961555004
Epoch 6/200
27/27 [==============================] - 63s 2s/step - loss: 0.1001 - acc: 0.9619 -

In [ ]:
#model.load_weights('/content/resnet50.h5')

In [ ]:
#model.evaluate_generator(generator=validation_generator,verbose=0) first resnet101v2 imagenet weights 

[0.14675359981548455, 0.9444444444444444]

In [ ]:
model.evaluate_generator(generator=validation_generator,verbose=0) #second one

[0.6837722642554209, 0.8611111111111112]

In [ ]:
test_generator.reset()
predictions=model.predict_generator(test_generator,verbose=1)
#print("loss: %.3f - acc: %.3f" % (predictions[0], predictions[1]))

610/610 [==============================] - 54s 88ms/step


In [ ]:
data=predictions.round(decimals=3)
data[0:6]

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
predicted_class_indices=np.argmax(predictions,axis=1)
labels = (train_generator.class_indices)
labels1 = dict((v,k) for k,v in labels.items())
pred = [labels1[k] for k in predicted_class_indices]

In [ ]:
pred[0:6]

['leaf_rust',
 'stem_rust',
 'leaf_rust',
 'healthy_wheat',
 'healthy_wheat',
 'stem_rust']

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":pred})


In [ ]:
def get_ids(filenames):
  for i in filenames:
    k=i.split('/')
    kk=k[1].split('.')
    return kk[0]

In [ ]:
results['Filename']=results.apply(lambda x: get_ids(x),axis=1)

In [ ]:
leaf_rust=pd.Series(range(610),name='leaf_rust',dtype=np.float32)
stem_rust=pd.Series(range(610),name='stem_rust',dtype=np.float32)
healthy_wheat=pd.Series(range(610),name='healthy_wheat',dtype=np.float32)
sub=pd.concat([healthy_wheat,leaf_rust,stem_rust],axis=1)
sub.shape

(610, 3)

In [ ]:
for i in tqdm(range(0,len(data))):
  sub.loc[i]=data[i]

100%|██████████| 610/610 [00:00<00:00, 5884.08it/s]


In [ ]:
sub["ID"]=results['Filename']

In [ ]:
sub.head(4)

,healthy_wheat,leaf_rust,stem_rust,ID
0,0.0,1.0,0.0,008FWT
1,0.0,0.0,1.0,00AQXY
2,0.0,1.0,0.0,01OJZX
3,1.0,0.0,0.0,07OXKK


In [ ]:
submit=sub[['ID','leaf_rust','stem_rust','healthy_wheat']]

In [ ]:
submit.head(6)

,ID,leaf_rust,stem_rust,healthy_wheat
0,008FWT,1.0,0.0,0.0
1,00AQXY,0.0,1.0,0.0
2,01OJZX,1.0,0.0,0.0
3,07OXKK,0.0,0.0,1.0
4,085IEC,0.0,0.0,1.0
5,08O2YE,0.0,1.0,0.0


In [ ]:
submit.to_csv('resnet101v2_fourth_submission.csv',index=False)